## MaskRCNN code base

In [1]:
import numpy as np

Understanding the anchor generating function

In [2]:
feature_shapes = 32
feature_strides = 512//32
anchor_sizes = [39,46,52,58,65]
anchor_stride = 1

In [3]:
x = np.arange(0, feature_shapes, anchor_stride) * feature_strides
y = np.arange(0, feature_shapes, anchor_stride) * feature_strides
x, y = np.meshgrid(x, y)

In [4]:
x.shape # The x coordinate of the lower left corner
# of each anchor

(32, 32)

In [5]:
width, x = np.meshgrid(anchor_sizes, x)
height, y = np.meshgrid(anchor_sizes, y)

In [8]:
np.shape(x) #The x coordinate of each anchor is still preserved
# but their positions are flattened, and we defined anchors of
# 5 different sizes at each position

(1024, 5)

In [6]:
# Reshape indices and shapes
x = x.reshape((-1, 1))
y = y.reshape((-1, 1))
width = width.flatten().reshape((-1, 1))
height = height.flatten().reshape((-1, 1))

In [7]:
x.shape

(5120, 1)

In [11]:
bbox_centers = np.concatenate((y, x), axis=1)
bbox_shapes = np.concatenate((height, width), axis=1)

In [12]:
bbox_centers.shape

(5120, 2)

In [13]:
bboxes = np.concatenate((bbox_centers - bbox_shapes / 2,
                        bbox_centers + bbox_shapes / 2), axis=1)

In [14]:
bboxes.shape # 2 corners of each anchor generated

(5120, 4)

In [15]:
# Anchors are created for each feature map
anchors = []
anchors.append(bboxes)
print('Num of generated anchors:\t', len(bboxes))

anchors = np.concatenate(anchors, axis=0)

Num of generated anchors:	 5120


Understanding proposal generating function (given RPN outputs)

In [16]:
rpn_bbox = np.random.normal(scale=5.0, size=(50, 32*32*5, 4))
rpn_probs = np.random.uniform(size=(50, 32*32*5, 1))
rpn_probs = np.concatenate((1-rpn_probs, rpn_probs), axis=2)

In [18]:
np.all(rpn_probs[:, :, 0]+rpn_probs[:, :, 1] == 1.0)

True

In [19]:
batchlen = 50
proposal_count = 20
mode = 'pixelwise'

proposals = np.zeros((batchlen, proposal_count, 4))
origanchors = np.zeros((batchlen,proposal_count,4))

In [20]:
pred_scores = rpn_probs[1]
pred_deltas = rpn_bbox[1]
# Find where predicted positive (foreground) boxes
positive_idxs = np.where(np.argmax(pred_scores, axis=-1)==1)[0]

In [30]:
positive_anchors = anchors[positive_idxs]

In [31]:
positive_anchors.shape

(2588, 4)

In [32]:
proposals = np.random.randint(0, 512, size=(batchlen, proposal_count, 4)).astype(float)

In [39]:
proposal = proposals[2]
proposal = proposal/512

In [40]:
proposal.shape

(20, 4)